In [160]:
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import ipywidgets as widgets

%matplotlib inline

In [161]:
df = None
last = datetime.now() - timedelta(hours=1)


def get_data():
    """
    Get the latest data. Just return the cached copy
    if less than one hour has elapsed.
    """
    global last, df
    now = datetime.now()
    if now - last < timedelta(hours=1):
        return df

    last = now    
    start = date(2020, 1, 22)
    end = date.today()

    df = pd.DataFrame()
    while start <= end:
        try:
            df_day = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{start.month:02d}-{start.day:02d}-{start.year}.csv')
            df_day['Date'] = start
            df = df.append(df_day)
        except Exception:
            pass
        start += timedelta(days=1)
    
    del df['Last Update']
    df = df.replace({'Country/Region': {'Mainland China': 'China'}})
    df.fillna({'Province/State': ''}, inplace=True)
    for col in ['Confirmed', 'Deaths', 'Recovered']:
        df.fillna({col: 0}, inplace=True)
    return df

In [162]:
df = get_data()
df.tail(5)

,Province/State,Country/Region,Confirmed,Deaths,Recovered,Date,Latitude,Longitude
244,"Virgin Islands, U.S.",US,1.0,0.0,0.0,2020-03-14,18.3358,-64.8963
245,Gibraltar,United Kingdom,1.0,0.0,1.0,2020-03-14,36.1408,-5.3536
246,From Diamond Princess,Australia,0.0,0.0,0.0,2020-03-14,35.4437,139.6380
247,West Virginia,US,0.0,0.0,0.0,2020-03-14,38.4912,-80.9545
248,,occupied Palestinian territory,0.0,0.0,0.0,2020-03-14,31.9522,35.2332


In [163]:
def aggregate_for_locations(df, locations, do_diffs = True, fields = None):
    if fields is None:
        fields = ['Confirmed', 'Deaths', 'Recovered']
    if isinstance(locations, str):
        locations = [locations]
    in_loc = df[df['Country/Region'].isin(locations)]
    cols = ['Date']
    cols.extend(fields)
    result = in_loc[cols].groupby('Date').sum()
    if do_diffs:
        for f in fields:
            result[f] = result[f].diff()
            result.fillna({f: 0}, inplace=True)
    return result


In [164]:
def plot_time_series(df, title):
    fig, ax = plt.subplots()
    ax.plot_date(df.index, df, 'v-')
    ax.xaxis.set_minor_locator(dates.WeekdayLocator(byweekday=(1), interval=1))
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%d\n%a'))
    ax.xaxis.grid(True, which="minor")
    ax.yaxis.grid()
    ax.xaxis.set_major_locator(dates.MonthLocator())
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n\n\n%b\n%Y'))
    ax.legend(df.columns, loc='upper left', shadow=True)
    ax.set_title(title)
    plt.tight_layout()
    return plt

In [165]:
locations = sorted(list(df['Country/Region'].unique()))

In [168]:
picker = widgets.SelectMultiple(
                        options=locations,
                        value=['US'],
                        description='Locations'
                )

In [169]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, do_diffs=False), title='Cumulative'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(160,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [170]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location), title='New'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(160,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>